In [ ]:
%cd ../..
%pip install .
import time
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# other1 = Car on intersection,
#             facing Range(50, 135) deg relative to ego.heading

# other2 = Car on intersection,
#             facing -1 * Range(50, 135) deg relative to ego.heading

# require abs(relative heading of other1 from other2) > 100 deg
# require (distance from ego to intersectionRegion) < 10

In [ ]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'car'
) as cars

select *
from cars as t1
join cars as t2 on t1.cameraId = t2.cameraId
join Cameras as cam on t1.cameraId = Cameras.id
where
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) >= -15 AND
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) <= 15 AND
  DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 50 AND
  viewAngle(t1.traj, cam.egoHeading, cam.ego_Translation, cam.timestamp) < 135 AND
  DISTANCE(cam.egoTranslation, t2.centroid, cam.timestamp) < 50 AND
  viewAngle(t2.traj, cam.egoHeading, cam.ego_Translation, cam.timestamp) < 135 AND
  contained(t1.traj, road_segment_type("intersection"), cam.timestamp) and
  contained(t2.traj, road_segment_type("intersection"), cam.timestamp) and
  (facingRelative(t1.heading, cam.egoHeading, cam.timestamp) < -50 ANd
  facingRelative(t1.heading, cam.egoHeading, cam.timestamp) > -135) AND
  (facingRelative(t2.heading, cam.egoHeading, cam.timestamp) < -50 AND
  facingRelative(t2.heading, cam.egoHeading, cam.timestamp) > -135) AND
  DISTANCE(cam.egoTranslation, roadSection("intersection"), cam.timestamp) < 10 AND
  (facingRelative(t1.heading, t2.heading, cam.timestamp) > 100 OR
  facingRelative(t1.heading, t2.heading, cam.timestmap) < -100)

"""

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

def show_filenames(w, key_and_timestamps):
    itemids = []
    ts = []
    for i in range(len(key_and_timestamps)):
        itemids.append(key_and_timestamps[i][0])
        ts.append(key_and_timestamps[i][-1])
    # clean up the filenames
    fnames = set(w.get_filenames_by_itemids_and_timestamps(itemids, ts))
    fnames = [fname.split("/")[-1] for fname in fnames]
    return fnames

def show_images_by_filenames(filenames):
    folder = "./data/experiment_data/"
    for name in filenames:
        fname = folder + name
        print(fname)
        plt.figure()
        plt.imshow(mpimg.imread(fname))


In [ ]:
### Prepare the world for queries ###
from apperception.new_world import *
# directly ingest the mini dataset and boston roadnetwork if needed
# from apperception.new_db import database
# database.reset()
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F
world = world.filter(lambda obj: obj.object_type == 'vehicle.car')
def pred(obj1, obj2, cam):
    return (
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and
        F.distance(cam.ego, obj1, cam.timestamp) < 50 and
        F.viewAngle(obj1, cam.ego, cam.timestamp) < 135 and
        F.distance(cam.ego, obj2, cam.timestamp) < 50 and
        F.viewAngle(obj2, cam.ego, cam.timestamp) < 135 and
        F.contained(obj1.traj, F.road_segment("intersection"), cam.timestamp) and
        F.contained(obj2.traj, F.road_segment("intersection"), cam.timestamp) and
        (F.facing_relative(obj1, cam.ego, cam.timestamp) >= 50 or
            F.facing_relative(obj1, cam.ego, cam.timestamp) <= 135) and
        (F.facing_relative(obj2, cam.ego, cam.timestamp) < -50 or
            F.facing_relative(obj2, cam.ego, cam.timestamp) > -135) and
        F.distance(cam.ego, roadSection(obj1.traj, cam.timestamp)) < 10 and
        (F.facing_relative(obj1, obj2, cam.timestamp) > 100 or
            F.facing_relative(obj1, obj2, cam.timestamp) < -100)
    )
world2 = world.filter("lambda obj1, obj2, cam: F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and " + 
        "F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and " + 
        "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
        "F.view_angle(obj1, cam.ego, cam.timestamp) < 135 and " +
        "F.distance(cam.ego, obj2, cam.timestamp) < 50 and " +
        "F.view_angle(obj2, cam.ego, cam.timestamp) < 135 and " +
        "F.contained(obj1.traj, F.road_segment('intersection'), cam.timestamp) and " +
        "F.contained(obj2.traj, F.road_segment('intersection'), cam.timestamp) and " +
        "(F.facing_relative(obj1, cam.ego, cam.timestamp) >= 50 or " +
            "F.facing_relative(obj1, cam.ego, cam.timestamp) <= 135) and " +
        "(F.facing_relative(obj2, cam.ego, cam.timestamp) < -50 or " +
            "F.facing_relative(obj2, cam.ego, cam.timestamp) > -135) and " +
        "F.distance(cam.ego, obj1, cam.timestamp) < 10 and " +
        "F.distance(cam.ego, obj2, cam.timestamp) < 10 and " +
        "(F.facing_relative(obj1, obj2, cam.timestamp) > 100 or " +
            "F.facing_relative(obj1, obj2, cam.timestamp) < -100)")

start = time.time()

# key = world2.get_id_and_time(num_joined_tables=2)

end = time.time()
print(format(end-start))

In [ ]:
# [*] The answer filename is in the result

world2 = world.filter("lambda obj1, obj2, cam: "+ # F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and " + 
#         "F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and " + 
        "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
#         "F.view_angle(obj1, cam.ego, cam.timestamp) < 135 and " +
        "F.distance(cam.ego, obj2, cam.timestamp) < 50 and " +
#         "F.view_angle(obj2, cam.ego, cam.timestamp) < 135 and " +
        "F.contained(obj1.traj, F.road_segment('intersection'), cam.timestamp) and " +
        "F.contained(obj2.traj, F.road_segment('intersection'), cam.timestamp)"
#         "(F.facing_relative(obj1, cam.ego, cam.timestamp) < -50 or " +
#             "F.facing_relative(obj1, cam.ego, cam.timestamp) > -135) and " +
#         "(F.facing_relative(obj2, cam.ego, cam.timestamp) < -50 or " +
#             "F.facing_relative(obj2, cam.ego, cam.timestamp) > -135) and " +
#         "F.distance(cam.ego, obj1, cam.timestamp) < 10 and " +
#         "F.distance(cam.ego, obj2, cam.timestamp) < 10 and " +
#         "(F.facing_relative(obj1, obj2, cam.timestamp) > 100 or " +
#             "F.facing_relative(obj1, obj2, cam.timestamp) < -100)"
        )

key = world2.get_id_and_time(num_joined_tables=2)
"n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118112404.jpg" in show_filenames(world, key)

In [ ]:
# [*] The answer filename is NOT in the result

world2 = world.filter("lambda obj1, obj2, cam: F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and " + 
        "F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and " + 
        "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
#         "F.view_angle(obj1, cam.ego, cam.timestamp) < 135 and " +
        "F.distance(cam.ego, obj2, cam.timestamp) < 50 and " +
#         "F.view_angle(obj2, cam.ego, cam.timestamp) < 135 and " +
        "F.contained(obj1.traj, F.road_segment('intersection'), cam.timestamp) and " +
        "F.contained(obj2.traj, F.road_segment('intersection'), cam.timestamp)"
#         "(F.facing_relative(obj1, cam.ego, cam.timestamp) < -50 or " +
#             "F.facing_relative(obj1, cam.ego, cam.timestamp) > -135) and " +
#         "(F.facing_relative(obj2, cam.ego, cam.timestamp) < -50 or " +
#             "F.facing_relative(obj2, cam.ego, cam.timestamp) > -135) and " +
#         "F.distance(cam.ego, obj1, cam.timestamp) < 10 and " +
#         "F.distance(cam.ego, obj2, cam.timestamp) < 10 and " +
#         "(F.facing_relative(obj1, obj2, cam.timestamp) > 100 or " +
#             "F.facing_relative(obj1, obj2, cam.timestamp) < -100)"
        )

key = world2.get_id_and_time(num_joined_tables=2)
"n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118112404.jpg" in show_filenames(world, key)

# Database debugging

In [ ]:
from apperception.new_db import database
# database.reset()
from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

In [ ]:
database.cursor.execute(
"""
select * from cameras where  position('n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118112404.jpg' in filename) > 0 limit 5;
""")
database.cursor.fetchall()

In [ ]:
database.cursor.execute(
"""
select roadDirection(Cameras.egoTranslation, Cameras.timestamp) from cameras where cameras.cameraid = 'scene-0757' AND cameras.timestamp = '2018-08-30 12:25:18.112404+00';
""")
database.cursor.fetchall()

In [ ]:
database.cursor.execute(
"""
select Cameras.egoHeading from cameras where cameras.cameraid = 'scene-0757' AND cameras.timestamp = '2018-08-30 12:25:18.112404+00';
"""
)
database.cursor.fetchall()

In [ ]:
database.cursor.execute(
"""
select facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.timestamp)) from cameras where cameras.cameraid = 'scene-0757' AND cameras.timestamp = '2018-08-30 12:25:18.112404+00';
""")
database.cursor.fetchall()

# Find camera segment_id and heading

In [ ]:
filename = "n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118112404.jpg"

def find_segmentid_and_heading_from_image(filename):
    database.cursor.execute(
    f"""
    select * from cameras where  position('{filename}' in filename) > 0 limit 5;
    """)
    camera_id = database.cursor.fetchall()[0][0]
    # print(camera_id)
    database.cursor.execute(
    f"""
    select ST_asText(egoTranslation) from cameras where cameras.cameraid = '{camera_id}' limit 1;
    """)
    
    ego_translation = database.cursor.fetchall()[0][0]
    # print(ego_translation)
    database.cursor.execute(
    f"""
    select segmentid, DEGREES(heading) from segment, ST_GeomFromText('{ego_translation}') as point order by ST_distance(segmentLine, point) asc limit 1;
    """)
    
    out = database.cursor.fetchall()
    
    segment_id = out[0][0]
    heading = out[0][1]
    return segment_id, heading

In [ ]:
filenames = ["n008-2018-09-18-12-07-26-0400__CAM_FRONT__1537287358412404.jpg", "n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118112404.jpg", "n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118612404.jpg", "n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127612404.jpg", "n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657128112404.jpg", "n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127112404.jpg", "n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657119112404.jpg"]
df_dict = {"filename": [], "segment_id": [], "heading": []}
for filename in filenames:
    segment_id, heading = find_segmentid_and_heading_from_image(filename)
    df_dict["filename"].append(filename)
    df_dict["segment_id"].append(segment_id)
    df_dict["heading"].append(heading)


In [ ]:
import pandas as pd
df = pd.DataFrame(df_dict)
display(df)
df.to_csv("fig-13.csv", index=False)